In [4]:
! pip install psycopg2-binary --user

import pandas as pd
import psycopg2
import numpy as np
from getpass import getpass

In [6]:
# connect to database
connection = psycopg2.connect(
    database  = "postgres",
    user      = "postgres",
    password  = getpass(),
    host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
    port      = '5432'
)


 ················


In [7]:
# cursor object
try:
    c = connection.cursor()
    print("Connected!")
except:
    print("Connection problem chief!")

Connected!


In [8]:
c.execute("SELECT movie_id, username FROM reviews")

In [9]:
result = c.fetchall()
result

[(None, 'me')]

In [10]:
df = pd.DataFrame(result, columns = ['movieid', 'userid'])
df.head()

,movieid,userid
0,None,me


In [11]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'

In [12]:
df2 = pd.read_table(url, sep='\t')

C:\Users\Cooper\Anaconda3\envs\Mike\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
C:\Users\Cooper\Anaconda3\envs\Mike\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df2['movieid'] =[i[2:] for i in df2['tconst']]

In [14]:
df2['movieid'] = df2['movieid'].astype(int)

In [15]:
df2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,movieid
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",2
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",3
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",4
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",5


In [16]:
df = df.merge(df2, how='inner', on='movieid')

ValueError: You are trying to merge on object and int32 columns. If you wish to proceed you should use pd.concat

In [ ]:
df.head()

In [16]:
df.movieid.value_counts()

105112     100
111282     100
80678      100
1261862    100
93894      100
50271      100
29605      100
108065     100
95031      100
88960      100
341376     100
1606389    100
83987      100
82269      100
1319694    100
23582      100
488164     100
1595656    100
60138      100
51196      100
2671706    100
376479     100
427339     100
490579     100
420333     100
22254      100
46896      100
3174376    100
40202      100
93207      100
          ... 
8315092      1
4946602      1
825348       1
114390       1
6613848      1
73653        1
191384       1
389558       1
356774       1
5591458      1
246717       1
7597790      1
89802        1
88900        1
207776       1
65212        1
1237913      1
94400        1
225845       1
53135        1
7686386      1
97998        1
28571        1
254324       1
193329       1
193029       1
108245       1
176653       1
156183       1
3294340      1
Name: movieid, Length: 107207, dtype: int64

In [49]:
movies = df.movieid.unique().tolist()

In [45]:
df['users'] = df['userid'].astype('str')

In [50]:
users = df.userid.unique().tolist()
len(users)

346407

In [27]:
df.dtypes

movieid     int64
userid     object
users      object
dtype: object

In [41]:
# ! sudo su
! yum update -y
! yum -y install python-pip
! python -V

]0;root@ip-172-16-97-236:/home/ec2-user/SageMaker/Groa[root@ip-172-16-97-236 Groa]# ^C
]0;root@ip-172-16-97-236:/home/ec2-user/SageMaker/Groa[root@ip-172-16-97-236 Groa]# Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
You need to be root to perform this command.
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
You need to be root to perform this command.
Python 3.6.5 :: Anaconda, Inc.


In [43]:
! which pip

/home/ec2-user/anaconda3/envs/python3/bin/pip


In [51]:
# ! python -m pip install tqdm
# # ! python -c 'import tqdm'
# ! python -m pip install gensim
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')



# shuffle usernames
random.shuffle(users)

# extract 90% of users
users_train = [users[i] for i in range(round(0.9*len(users)))]

# split data into train and validation set
train_df = df[df['users'].isin(users_train)]
validation_df = df[~df['users'].isin(users_train)]

train_df.head()

,movieid,userid,users,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,72777,nmadhuk3,nmadhuk3,tt0072777,movie,Chhoti Si Baat,Chhoti Si Baat,0,1976,\N,123,"Comedy,Romance"
1,72777,mishu_mausam,mishu_mausam,tt0072777,movie,Chhoti Si Baat,Chhoti Si Baat,0,1976,\N,123,"Comedy,Romance"
2,72777,Kaaliyaa,Kaaliyaa,tt0072777,movie,Chhoti Si Baat,Chhoti Si Baat,0,1976,\N,123,"Comedy,Romance"
3,72777,Peter_Young,Peter_Young,tt0072777,movie,Chhoti Si Baat,Chhoti Si Baat,0,1976,\N,123,"Comedy,Romance"
4,72777,bluegreensalad,bluegreensalad,tt0072777,movie,Chhoti Si Baat,Chhoti Si Baat,0,1976,\N,123,"Comedy,Romance"


In [46]:
# list to capture watch history of the users
watched_train = []

# populate the list with the movie ids
for i in tqdm(users_train):
    temp = train_df[train_df["users"] == i]["movieid"].tolist()
    watched_train.append(temp)

100%|██████████| 311766/311766 [3:49:23<00:00, 22.65it/s]  


In [73]:
# list to capture purchase history of the customers
watched_val = []

# populate the list with the product codes
for i in tqdm(validation_df['users'].unique()):
    temp = validation_df[validation_df["users"] == i]["movieid"].tolist()
    watched_val.append(temp)

100%|██████████| 34641/34641 [04:53<00:00, 117.83it/s]


In [68]:
watched_train = [[str(x) for x in lst] for lst in watched_train]

In [73]:
watched_train[0]

['6110504']

In [74]:
watched_val = [[str(x) for x in lst] for lst in watched_val]

In [75]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watched_train, progress_per=200)

model.train(watched_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(8195290, 9259500)

In [76]:
# save word2vec model
model.save("word2vec_2.model")

In [40]:
import gensim

model = gensim.models.Word2Vec.load("word2vec_2.model")

In [ ]:
model.init_sims(replace=True)

In [ ]:
print(model)

In [41]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

(49385, 100)

In [ ]:
# ! sudo pip install umap-learn

In [66]:
train_df['movieid'] = train_df['movieid'].astype(str)
train_df['movieid'][0]

'72777'

In [67]:
movies = train_df[["movieid", "primaryTitle"]]

# remove duplicates
movies.drop_duplicates(inplace=True, subset='movieid', keep="last")

# create product-ID and product-description dictionary
movies_dict = movies.groupby('movieid')['primaryTitle'].apply(list).to_dict()

In [68]:
# test the dictionary
movies_dict['105112']



['Patriot Games']

In [69]:
def similar_movies(v, n = 6):
    
    # extract most similar movies for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract title and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (movies_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [70]:
similar_movies(model['105112'])

[('How the Grinch Stole Christmas', 0.730616569519043),
 ('Split Second', 0.7187882661819458),
 ('The Final Countdown', 0.7176185846328735),
 ("Livin' Large!", 0.7173690795898438),
 ('Hawks', 0.7129511833190918),
 ('Manhunter', 0.7002811431884766)]

In [71]:
def aggregate_vectors(movies):
    movie_vec = []
    for i in movies:
        try:
            movie_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(movie_vec, axis=0)


In [80]:
len(watched_val[2])

597

In [81]:
aggregate_vectors(watched_val[2]).shape

(100,)

In [82]:
similar_movies(aggregate_vectors(watched_val[2]))

[('Our Miss Fred', 0.9716984629631042),
 ('Lovefilm', 0.9665443301200867),
 ('Gene-X', 0.9657200574874878),
 ('O Pátio das Cantigas', 0.9629855751991272),
 ('Mercano the Martian', 0.9613674283027649),
 ("Hitler's Grave", 0.96126788854599)]

In [83]:
similar_movies(aggregate_vectors(watched_val[2][-10:]))

[('South Sea Woman', 0.9722521305084229),
 ('The Joyless Street', 0.969933807849884),
 ('Ladies in Retirement', 0.9696546792984009),
 ('The Singing Kid', 0.9678635597229004),
 ('Transatlantic Merry-Go-Round', 0.9676827192306519),
 ('Are We Civilized?', 0.9664505124092102)]